In [ ]:
import pandas as pd
import shapash

In [2]:

import numpy as np
import matplotlib.pyplot as plt; plt.rcdefaults()
from sklearn.metrics import confusion_matrix,accuracy_score,roc_curve,classification_report,auc
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
import seaborn as sns
import plotly.express as px

from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import RFECV
#from yellowbrick.model_selection import RFECV as YERFECV
import sklearn

from sklearn.linear_model import LogisticRegression

#from ydata_profiling import ProfileReport

#import shap
#shap.initjs()



In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn import preprocessing

# HEART DATA

# BASIC UNDERSTANDING OF THE DATASET

In [4]:
heart = pd.read_csv('data/bigheart.csv')
print(heart.shape)
#print(heart.head())
#heart.info()
from collections import Counter
print(Counter(heart['HeartDisease']))  
heart.describe()

(918, 12)
Counter({1: 508, 0: 410})


,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


## One hot encoding

In [5]:
heart2 = pd.get_dummies(heart, columns=['Sex', 'ChestPainType', 'RestingECG','ExerciseAngina',\
                                        'ST_Slope'])

# CREATE X AND y 

In [6]:
X = heart2.drop('HeartDisease',axis=1)
y = heart2['HeartDisease']
print(X.shape)


(918, 20)


# GET THE NAMES OF COLUMNS TO USE LATER TO DISPLAY REGRESSION COEFFICIENTS

In [7]:
column_headers = list(X.columns.values)
print("The Column Header :", column_headers)

The Column Header : ['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak', 'Sex_F', 'Sex_M', 'ChestPainType_ASY', 'ChestPainType_ATA', 'ChestPainType_NAP', 'ChestPainType_TA', 'RestingECG_LVH', 'RestingECG_Normal', 'RestingECG_ST', 'ExerciseAngina_N', 'ExerciseAngina_Y', 'ST_Slope_Down', 'ST_Slope_Flat', 'ST_Slope_Up']


# DATA PRE-PROCESSING ENDS HERE

# -----------------------------------------------------

 # MODELING

# SPLIT INTO TESTING AND TRAINING SET USING STRATIFY

In [8]:

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.33, random_state=42, stratify=y)
print(len(y_train))
X_testorig=X_test.copy()
print(type(y_train))

# THIS IS NEEDED FOR THE FIT FUNCTION OF THE ESTIMATORS. THE SERIES NEEDS TO BE CONVERTED TO 
#A 1-D ARRAY.
y_train= y_train.to_numpy()
y_test= y_test.to_numpy()
y_testorig=y_test.copy()



615
<class 'pandas.core.series.Series'>


# SHAPASH

In [ ]:
from shapash.explainer.smart_explainer import SmartExplainer

from sklearn.ensemble import RandomForestClassifier
model2 = RandomForestClassifier(n_estimators=200, criterion='entropy', max_features=
                                'sqrt',random_state=12,max_depth=5)
sw=model2.fit(X_train,y_train)
model2_predicted = model2.predict(X_test)


heart_features_dict={'Age': "patient age",
              'Sex_F': "Sex F",
              'Sex_M': 'Sex M',
              'ChestPainType_ASY': 'chest_pain_type ASY',
              'ChestPainType_ATA': 'chest_pain_type ATA',
              'ChestPainType_NAP': 'chest_pain_type NAP',
              'ChestPainType_TA' : 'chest_pain_typeTA',
               'RestingBP': 'resting blood pressure',
               'Cholesterol':'cholesterol',
               'FastingBS': 'fasting blood pressure',
               'RestingECG_LVH':'EKG LVH',
               'RestingECG_Normal':'EKG Normal',
               'RestingECG_ST': 'EKG ST',
               'MaxHR': 'max heart rate',
               'ExerciseAngina_N':'angina N',
               'ExerciseAngina_Y': 'angina Y',
               'Oldpeak':'oldpeak',
               'ST_Slope_Down':'ST_segment Down',
               'ST_Slope_Up': 'St Slope up',
               'ST_Slope_Flat': 'St Slope Flat'
               
              }
xpl = SmartExplainer(model=sw, features_dict=heart_features_dict)
xpl.compile(x=X_test) #KNN is not supported by shapash

app = xpl.run_app(title_story='Heart')

# FEATUREWIZ

In [17]:
from featurewiz import featurewiz
feats = featurewiz(heart2, 'HeartDisease',feature_engg='',corr_limit=0.9)

featurewiz has selected 0.9 as the correlation limit. Change this limit to fit your needs...
Skipping feature engineering since no feature_engg input...
Skipping category encoding since no category encoders specified in input...
#### Single_Label Binary_Classification problem ####
    Loaded train data. Shape = (918, 21)
#### Single_Label Binary_Classification problem ####
No test data filename given...
#######################################################################################
######################## C L A S S I F Y I N G  V A R I A B L E S  ####################
#######################################################################################
        No variables were removed since no ID or low-information variables found in data set
Removing 0 columns from further processing since ID or low information variables
#######################################################################################
#####  Searching for Uncorrelated List Of Variables (SULOV) in 20 f